In [1]:
import numpy as np
from typing import Literal, List
import gsw

In [2]:
DBAR_TO_PSI = 1.450377
PSI_TO_DBAR = 0.6894759
OXYGEN_PHASE_TO_VOLTS = 39.457071
KELVIN_OFFSET_0C = 273.15
KELVIN_OFFSET_25C = 298.15
OXYGEN_MLPERL_TO_MGPERL = 1.42903
OXYGEN_MLPERL_TO_UMOLPERKG = 44.660
# taken from https://blog.seabird.com/ufaqs/what-is-the-difference-in-temperature-expressions-between-ipts-68-and-its-90/ # pylint: disable=line-too-long
ITS90_TO_IPTS68 = 1.00024
# micro moles of nitrate to milligrams of nitrogen per liter
UMNO3_TO_MGNL = 0.014007
# [J K^{-1} mol^{-1}] Gas constant, NIST Reference on Constants retrieved 10-05-2015
R = 8.3144621
# [Coulombs mol^{-1}] Faraday constant, NIST Reference on Constants retrieved 10-05-2015
F = 96485.365

In [3]:
class TemperatureCoefficients:
    """
    :param a0: coefficient
    :param a1: coefficient
    :param a2: coefficient
    :param a3: coefficient
    """

    a0: float
    a1: float
    a2: float
    a3: float

class PressureCoefficients:
    """
    :param pa0: coefficient
    :param pa1: coefficient
    :param pa2: coefficient
    :param ptca0: coefficient
    :param ptca1: coefficient
    :param ptca2: coefficient
    :param ptcb0: coefficient
    :param ptcb1: coefficient
    :param ptcb2: coefficient
    :param ptempa0: coefficient
    :param ptempa1: coefficient
    :param ptempa2: coefficient
    """

    pa0: float
    pa1: float
    pa2: float
    ptca0: float
    ptca1: float
    ptca2: float
    ptcb0: float
    ptcb1: float
    ptcb2: float
    ptempa0: float
    ptempa1: float
    ptempa2: float

class ConductivityCoefficients:
    """
    :param g: coefficient
    :param h: coefficient
    :param i: coefficient
    :param j: coefficient
    :param cpcor: compressibility coefficient
    :param ctcor: coefficient
    :param wbotc: bridge oscillator temperature coefficient see the
        37 Manual: https://www.seabird.com/asset-get.download.jsa?id=54627862348
    """

    g: float
    h: float
    i: float
    j: float
    cpcor: float
    ctcor: float
    wbotc: float

class Oxygen43Coefficients:
    """
    :param soc: linear scaling coefficient
    :param v_offset: voltage at zero oxygen signal
    :param tau_20: sensor time constant tau(T,P) at 20 C, 1 atmosphere, 0 PSU;
        slope term in calculation of tau(T,P)
    :param a: coefficient
    :param b: coefficient
    :param c: coefficient
    :param e: coefficient
    :param d0: tau(T,P) coefficient
    :param d1: tau(T,P) coefficient
    :param d2: tau(T,P) coefficient
    :param h1: hysteresis correction coefficient
    :param h2: hysteresis correction coefficient
    :param h3: hysteresis correction coefficient
    """

    soc: float
    v_offset: float
    tau_20: float
    a: float
    b: float
    c: float
    e: float
    d0: float  # TODO: the sensor lists a d0 coefficient, but it doesn't seem to be used?
    d1: float
    d2: float
    h1: float
    h2: float
    h3: float

In [4]:
class PAR_Coefficients:
    m: float
    b: float
    calicon: float #CalibrationConstant
    multiplier: float
    offset: float

class Altimeter_Coefficients:
    scalefactor: float
    offset: float

class FluroWetlab_Coefficients:
    scalefactor: float
    Vblank: float

class Turbidity_Coefficients:
    scalefactor: float
    darkvoltage: float

class Spar_Coefficients:
    conversionfactor: float
    ratiomultiplier: float



In [5]:
temp1_coefs = TemperatureCoefficients()
temp1_coefs.a0 = 4.33459406e-003
temp1_coefs.a1 = 6.28735680e-004
temp1_coefs.a2 = 2.05782221e-005
temp1_coefs.a3 = 1.68624541e-006

temp2_coefs = TemperatureCoefficients()
temp2_coefs.a0 = 4.34471154e-003
temp2_coefs.a1 = 6.38146193e-004
temp2_coefs.a2 = 2.04525401e-005
temp2_coefs.a3 = 1.60292340e-006


pres_coefs = PressureCoefficients()
pres_coefs.pa0 = -4.212464e+004
pres_coefs.pa1 = -1.972709e-001
pres_coefs.pa2 = 1.365930e-002

pres_coefs.ptca0 = 3.604100e-002
pres_coefs.ptca1 = 0.000000e+000
pres_coefs.ptca2 = 3.046128e+001

pres_coefs.ptcb0 = -3.791930e-004
pres_coefs.ptcb1 = 4.202260e-006
pres_coefs.ptcb2 = 3.386190e-00

pres_coefs.ptempa0 = 0.000000e+000
pres_coefs.ptempa1 = 1.279410e-002
pres_coefs.ptempa2 = -9.215740e+000


cond1_coefs = ConductivityCoefficients()
cond1_coefs.g = -9.94900904e+000
cond1_coefs.h = 1.42164741e+000
cond1_coefs.i = 1.26008747e-004
cond1_coefs.j = 6.16820694e-005
cond1_coefs.cpcor = -9.57000000e-008
cond1_coefs.ctcor = 3.2500e-006
cond1_coefs.wbotc = 0.00000000e+000

cond2_coefs = ConductivityCoefficients()
cond2_coefs.g = -9.77824825e+000
cond2_coefs.h = 1.31877668e+000
cond2_coefs.i = 2.26556605e-004
cond2_coefs.j = 6.59127902e-005
cond2_coefs.cpcor = -9.57000000e-008
cond2_coefs.ctcor = 3.2500e-006
cond2_coefs.wbotc = 0.00000000e+000


oxy1_coefs = Oxygen43Coefficients()
oxy1_coefs.soc = 4.7612e-001
oxy1_coefs.v_offset = -0.4891
oxy1_coefs.tau_20 = 1.7400
oxy1_coefs.a = -3.2939e-003
oxy1_coefs.b = 1.4617e-004
oxy1_coefs.c = -2.1408e-006
oxy1_coefs.e = 3.6000e-002
oxy1_coefs.d0 = 2.5826e+000
oxy1_coefs.d1 = 1.92634e-004
oxy1_coefs.d2 = -4.64803e-002
oxy1_coefs.h1 = -3.3000e-002
oxy1_coefs.h2 = 5.0000e+003
oxy1_coefs.h3 = 1.4500e+003

oxy2_coefs = Oxygen43Coefficients()
oxy2_coefs.soc = 5.4771e-001
oxy2_coefs.v_offset = -0.5020
oxy2_coefs.tau_20 = 1.2900
oxy2_coefs.a = -3.9126e-003
oxy2_coefs.b = 1.5415e-004
oxy2_coefs.c = -2.2575e-006
oxy2_coefs.e = 3.6000e-002
oxy2_coefs.d0 = 2.5826e+000
oxy2_coefs.d1 = 1.92634e-004
oxy2_coefs.d2 = -4.64803e-002
oxy2_coefs.h1 = -3.3000e-002
oxy2_coefs.h2 = 5.0000e+003
oxy2_coefs.h3 = 1.4500e+003

In [6]:
hexstrings = [
    "0CD08410767B80458F0CCA6F1107AE7D58A3987C41F0DEF2FFFFBE0000BB29512A089DB7004D638E5C91C465",
    "0CD0811076768045960CCA6F1107B27D48A3986C40F0DEF2FFFFBD0000BB29512A089DB7004D638F5C91C465",
    "0CD08710767B8045960CCA6D1107B17D48A3986C40F10EF0FFFFBE0000BB29512A089DB7004D63905D91C465",
    "0CD08710767A80458F0CCA6C1107AE7D58A3986C3FF16EE7FFFFBE0000BB29512A089DB7004D63915D91C465",
    "0CD08A10767680459E0CCA6F1107B27D58A3987C40F18EE3FFFFBE0000BB29512A089DB7004D63925D91C465",
    "0CD08410767A8045960CCA6C1107B17D58A3988C41F18EE3FFFFBE0000BB29512A089DB7004D63935D91C465",
    "0CD0811076798045960CCA6A1107B17D58A498AC41F18EE1FFFFBE0000BB29512A089DB7004D63945D91C465",
    "0CD08410767C80458F0CCA6A1107B07D48A398BC40F15ED4FFFFBE0000BB29512A089DB7004D63955D91C465",
    "0CD08710767A8045960CCA6F1107B17D48A498CC41F13ECEFFFFBE0000BB29512A089DB7004D63965D91C465",
    "0CD08110767980459D0CCA6F1107B07D48A398DC41F13ECEFFFFBD0000BB29512A089DB7004D63975D91C465"
]
bytes_list_array = [[hex_string[i:i+2] for i in range(0, len(hex_string), 2)] for hex_string in hexstrings]


hex_string = "0CD08410767B80458F0CCA6F1107AE7D58A3987C41F0DEF2FFFFBE0000BB29512A089DB7004D638E5C91C465"
bytes_list = [hex_string[i:i+2] for i in range(0, len(hex_string), 2)]

In [7]:
def hex_to_voltage(hex_value: str):
    clean_hex = hex_value.strip().upper()
    
    # Hex to Decimal
    N = int(clean_hex, 16)

    # Convert to volt
    V = 5 * (1 - (N / 4095))

    return V

def Time_elapsed(hexstrings, freq=24):
    """
    Computes elapsed time for a list of hex strings.
    Assumption: Each string = 1 measurement, starting at 0 s.
    
    :param hexstrings: List of hex strings (each string is one scan).
    :param freq: Sampling frequency in Hz (default 24 Hz).
    :return: List of elapsed times (in seconds).
    """
    dt = 1 / freq  # time interval per measurement
    elapsed = [i * dt for i in range(len(hexstrings))]
    return elapsed

In [8]:
def convert_temperature(
    temperature_counts_in: np.ndarray,
    coefs: TemperatureCoefficients,
    standard: Literal["ITS90", "IPTS68"] = "ITS90",
    units: Literal["C", "F"] = "C",
    use_mv_r: bool = False,
):
    """Returns the value after converting it to degrees C, ITS-90.

    Data is expected to be raw data from an instrument in A/D counts

    :param temperature_counts_in: temperature value to convert in A/D
        counts
    :param coefs: calibration coefficients for the temperature sensor
    :param standard: whether to use ITS90 or to use IPTS-68 calibration
        standard
    :param units: whether to use celsius or to convert to fahrenheit
    :param use_mv_r: true to perform extra conversion steps required by
        some instruments (check the cal sheet to see if this is required)

    :return: temperature val converted to ITS-90 degrees C
    """

    if use_mv_r:
        mv = (temperature_counts_in - 524288) / 1.6e007
        r = (mv * 2.900e009 + 1.024e008) / (2.048e004 - mv * 2.0e005)
        temperature_counts = r
    else:
        temperature_counts = temperature_counts_in

    log_t = np.log(temperature_counts)
    temperature = (
        1 / (coefs.a0 + coefs.a1 * log_t + coefs.a2 * log_t**2 + coefs.a3 * log_t**3)
    ) - KELVIN_OFFSET_0C

    if standard == "IPTS68":
        temperature *= ITS90_TO_IPTS68
    if units == "F":
        temperature = temperature * 9 / 5 + 32  # Convert C to F

    return temperature



def convert_pressure(
    pressure_count: np.ndarray,
    compensation_voltage: np.ndarray,
    coefs: PressureCoefficients,
    units: Literal["dbar", "psia"] = "psia",
):
    """Converts pressure counts to PSIA (pounds per square inch, abolute).

    pressure_count and compensation_voltage are expected to be raw data
    from an instrument in A/D counts

    :param pressure_count: pressure value to convert, in A/D counts
    :param compensation_voltage: pressure temperature compensation
        voltage, in counts or volts depending on the instrument
    :param coefs: calibration coefficients for the pressure sensor
    :param units: whether or not to use psia or dbar as the returned
        unit type

    :return: pressure val in PSIA
    """
    sea_level_pressure = 14.7

    t = (
        coefs.ptempa0
        + coefs.ptempa1 * compensation_voltage
        + coefs.ptempa2 * compensation_voltage**2
    )
    x = pressure_count - coefs.ptca0 - coefs.ptca1 * t - coefs.ptca2 * t**2
    n = x * coefs.ptcb0 / (coefs.ptcb0 + coefs.ptcb1 * t + coefs.ptcb2 * t**2)
    pressure = coefs.pa0 + coefs.pa1 * n + coefs.pa2 * n**2 - sea_level_pressure

    if units == "dbar":
        pressure *= PSI_TO_DBAR

    return pressure


def convert_conductivity(
    conductivity_count: np.ndarray,
    temperature: np.ndarray,
    pressure: np.ndarray,
    coefs: ConductivityCoefficients,
):
    """Converts raw conductivity counts to S/m.

    Data is expected to be raw data from instrument in A/D counts

    :param conductivity_count: conductivity value to convert, in A/D
        counts
    :param temperature: reference temperature, in degrees C
    :param pressure: reference pressure, in dbar
    :param coefs: calibration coefficient for the conductivity sensor

    :return: conductivity val converted to S/m
    """
    f = conductivity_count * np.sqrt(1 + coefs.wbotc * temperature) / 1000
    numerator = coefs.g + coefs.h * f**2 + coefs.i * f**3 + coefs.j * f**4
    denominator = 1 + coefs.ctcor * temperature + coefs.cpcor * pressure
    return numerator / denominator


def convert_sbe43_oxygen(
    voltage: np.ndarray,
    temperature: np.ndarray,
    pressure: np.ndarray,
    salinity: np.ndarray,
    coefs: Oxygen43Coefficients,
    apply_tau_correction: bool = False,
    apply_hysteresis_correction: bool = False,
    window_size: float = 1,
    sample_interval: float = 1,
):
    """Returns the data after converting it to ml/l.

    voltage is expected to be in volts, temperature in deg c, pressure
    in dbar, and salinity in practical salinity (PSU). All equation
    information comes from the June 2013 revision of the SBE43 manual

    :param voltage: SBE43 voltage
    :param temperature: temperature value converted to deg C
    :param pressure: Converted pressure value from the attached CTD, in
        dbar
    :param salinity: Converted salinity value from the attached CTD, in
        practical salinity PSU
    :param coefs: calibration coefficients for the SBE43 sensor
    :param apply_tau_correction: whether or not to run tau correction
    :param apply_hysteresis_correction: whether or not to run hysteresis
        correction
    :param window_size: size of the window to use for tau correction, if
        applicable, in seconds
    :param sample_interval: sample rate of the data to be used for tau
        correction, if applicable. In seconds.

    :return: converted Oxygen values, in ml/l
    """
    # start with all 0 for the dvdt
    dvdt_values = np.zeros(len(voltage))
    if apply_tau_correction:
        # Calculates how many scans to have on either side of our median
        # point, accounting for going out of index bounds
        scans_per_side = floor(window_size / 2 / sample_interval)
        for i in range(scans_per_side, len(voltage) - scans_per_side):
            ox_subset = voltage[i - scans_per_side : i + scans_per_side + 1]

            time_subset = np.arange(
                0, len(ox_subset) * sample_interval, sample_interval, dtype=float
            )

            result = stats.linregress(time_subset, ox_subset)

            dvdt_values[i] = result.slope

    correct_ox_voltages = voltage.copy()
    if apply_hysteresis_correction:
        # Hysteresis starts at 1 because 0 can't be corrected
        for i in range(1, len(correct_ox_voltages)):
            # All Equation info from APPLICATION NOTE NO. 64-3
            d = 1 + coefs.h1 * (np.exp(pressure[i] / coefs.h2) - 1)
            c = np.exp(-1 * sample_interval / coefs.h3)
            ox_volts = correct_ox_voltages[i] + coefs.v_offset

            prev_ox_volts_new = correct_ox_voltages[i - 1] + coefs.v_offset
            ox_volts_new = ((ox_volts + prev_ox_volts_new * c * d) - (prev_ox_volts_new * c)) / d
            ox_volts_final = ox_volts_new - coefs.v_offset
            correct_ox_voltages[i] = ox_volts_final

    oxygen = _convert_sbe43_oxygen(
        correct_ox_voltages,
        temperature,
        pressure,
        salinity,
        coefs,
        dvdt_values,
    )
    return oxygen

def _convert_sbe43_oxygen(
    voltage: np.ndarray,
    temperature: np.ndarray,
    pressure: np.ndarray,
    salinity: np.ndarray,
    coefs: Oxygen43Coefficients,
    dvdt_value: np.ndarray,
):
    """Returns the data after converting it to ml/l.

    voltage is expected to be in volts, temperature in deg c, pressure
    in dbar, and salinity in practical salinity (PSU). All equation
    information comes from the June 2013 revision of the SBE43 manual.
    Expects that hysteresis correction is already performed on the
    incoming voltage, if desired.

    :param voltage: SBE43 voltage
    :param temperature: temperature value converted to deg C
    :param pressure: Converted pressure value from the attached CTD, in
        dbar
    :param salinity: Converted salinity value from the attached CTD, in
        practical salinity PSU
    :param coefs: calibration coefficients for the SBE43 sensor
    :param dvdt_value: derivative value of voltage with respect to time
        at this point. Expected to be 0 if not using Tau correction

    :return: converted Oxygen value, in ml/l
    """

    # Oxygen Solubility equation constants, From SBE43 Manual Appendix A
    a0 = 2.00907
    a1 = 3.22014
    a2 = 4.0501
    a3 = 4.94457
    a4 = -0.256847
    a5 = 3.88767
    b0 = -0.00624523
    b1 = -0.00737614
    b2 = -0.010341
    b3 = -0.00817083
    c0 = -0.000000488682

    ts = np.log((KELVIN_OFFSET_25C - temperature) / (KELVIN_OFFSET_0C + temperature))
    a_term = a0 + a1 * ts + a2 * ts**2 + a3 * ts**3 + a4 * ts**4 + a5 * ts**5
    b_term = salinity * (b0 + b1 * ts + b2 * ts**2 + b3 * ts**3)
    c_term = c0 * salinity**2
    solubility = np.exp(a_term + b_term + c_term)

    # Tau correction
    tau = coefs.tau_20 * np.exp(coefs.d1 * pressure + coefs.d2 * (temperature - 20)) * dvdt_value

    soc_term = coefs.soc * (voltage + coefs.v_offset + tau)
    temp_term = 1.0 + coefs.a * temperature + coefs.b * temperature**2 + coefs.c * temperature**3
    oxygen = (
        soc_term
        * solubility
        * temp_term
        * np.exp((coefs.e * pressure) / (temperature + KELVIN_OFFSET_0C))
    )
    return oxygen

def convert_oxygen_to_umol_per_kg(ox_values: np.ndarray, potential_density: np.ndarray):
    """Converts given oxygen values to milligrams/kg.

    Note: Sigma-Theta is expected to be calculated via gsw_sigma0,
    meaning is it technically potential density anomaly. Calculating
    using gsw_rho(SA, CT, p_ref = 0) results in actual potential
    density, but this function already does the converison, so values
    will need to have 1000 subtracted from them before being passed into
    this function. The function is done this way to stay matching to
    Application Note 64, but the results of either method are identical.

    :param ox_values: oxygen values, already converted to ml/L
    :param potential_density: potential density (sigma-theta) values.
        Expected to be the same length as ox_values

    :return: oxygen values converted to milligrams/Liter
    """

    oxygen_umolkg = (ox_values * OXYGEN_MLPERL_TO_UMOLPERKG) / (potential_density + 1000)
    return oxygen_umolkg

def potential_density_from_t_s_p(
    temperature: np.ndarray,
    salinity: np.ndarray,
    pressure: np.ndarray,
    lon=0.0,
    lat=0.0,
    reference_pressure=0.0,
):
    """Derive potential density from measured temperature, salinity, and
    pressure.

    :param temperature: Measure temperature, in degrees C
    :param salinity: Measured salinity, in practical salinity units
    :param pressure: Measured pressure, in decibars
    :param lon: Longitude
    :param lat: Latitude
    :param reference_pressure: Reference pressure in decibars. Defaults
        to 0.0.

    :return: Potential density in kg/m^3
    """

    absolute_salinity = gsw.SA_from_SP(salinity, pressure, lon, lat)
    conservative_temperature = gsw.CT_from_t(absolute_salinity, temperature, pressure)
    potential_density = (
        gsw.rho(absolute_salinity, conservative_temperature, reference_pressure) - 1000
    )
    return potential_density

def depth_from_pressure(
    pressure_in: np.ndarray,
    latitude: float,
    depth_units: Literal["m", "ft"] = "m",
    pressure_units: Literal["dbar", "psi"] = "dbar",
):
    """Derive depth from pressure and latitude.

    :param pressure: Numpy array of floats representing pressure, in
        dbar or psi
    :param latitude: Latitude (-90.0 to 90.0)
    :param depth_units: 'm' for meters, 'ft' for feet. Defaults to 'm'.
    :param pressure_units: 'dbar' for decibars, 'psi' for PSI. Defaults
        to 'dbar'.

    :return: A numpy array representing depth in meters or feet
    """
    pressure = pressure_in.copy()
    if pressure_units == "psi":
        pressure /= DBAR_TO_PSI

    depth = -gsw.z_from_p(pressure, latitude)

    if depth_units == "ft":
        depth *= 3.28084

    return depth


In [9]:
F0 = 1000
temperature_counts_in1 = np.array(F0/(int(bytes_list[0],16)*256 + int(bytes_list[1],16) + int(bytes_list[2],16)/256))
temperature1 = convert_temperature(temperature_counts_in1,coefs= temp1_coefs)
print("Temp_1:",temperature1,"°C")

temperature_counts_in2 = np.array(F0/(int(bytes_list[9],16)*256 + int(bytes_list[10],16) + int(bytes_list[11],16)/256))
temperature2 = convert_temperature(temperature_counts_in2,coefs= temp2_coefs)
print("Temp_2:",temperature2,"°C")


Temp_1: 3.561563969710619 °C
Temp_2: 3.561044266345732 °C


In [10]:
pressure_counts = np.array((int(bytes_list[6],16)*256 + int(bytes_list[7],16) + int(bytes_list[8],16)/256))
compensation_voltage = np.array(1)

print(bytes_list[6],bytes_list[7],bytes_list[8])
print((int(bytes_list[6],16)),(int(bytes_list[7],16)),(int(bytes_list[8],16)))
print("Pressure_count:", pressure_counts)
print("Compensation_voltage:", compensation_voltage)


pressure = convert_pressure(pressure_counts,compensation_voltage,coefs= pres_coefs)
print("Pressure:", pressure,"db")

80 45 8F
128 69 143
Pressure_count: 32837.55859375
Compensation_voltage: 1
Pressure: -42139.332086015915 db


In [11]:
pressure_test = 1.149
conductivity_count = np.array((int(bytes_list[3],16)*256 + int(bytes_list[4],16) + int(bytes_list[5],16)/256))
conductifity1 = convert_conductivity(conductivity_count,temperature1,pressure_test,coefs= cond1_coefs)
print("Conductivity_1:", conductifity1,"mS/cm")

conductivity_count = np.array((int(bytes_list[12],16)*256 + int(bytes_list[13],16) + int(bytes_list[14],16)/256))
conductifity2 = convert_conductivity(conductivity_count,temperature2,pressure_test,coefs= cond2_coefs)
print("Conductivity_2:", conductifity2,"mS/cm")

Conductivity_1: 15.330789300015969 mS/cm
Conductivity_2: 15.32989468343113 mS/cm


In [12]:
import gsw  # TEOS-10 library
voltage1 = np.array([hex_to_voltage(hex_string[30:33])])
voltage2 = np.array([hex_to_voltage(hex_string[33:36])])
salinity1 = np.array(gsw.SP_from_C(conductifity1, temperature1, pressure_test))
salinity2 = np.array(gsw.SP_from_C(conductifity2, temperature2, pressure_test))


oxygen1 = convert_sbe43_oxygen(voltage1,temperature1,pressure_test,salinity1,coefs= oxy1_coefs)
oxygen2 = convert_sbe43_oxygen(voltage2,temperature2,pressure_test,salinity2,coefs= oxy2_coefs)
print("Oxygen1:", oxygen1,"ml/l")
print("Oxygen2:", oxygen2,"ml/l")


potential_density1 = potential_density_from_t_s_p(temperature1,salinity1,pressure_test)
potential_density2 = potential_density_from_t_s_p(temperature2,salinity2,pressure_test)

oxygen3 = convert_oxygen_to_umol_per_kg(np.array(oxygen1), np.array(potential_density1)) *1000
oxygen4 = convert_oxygen_to_umol_per_kg(np.array(oxygen2), np.array(potential_density2)) *1000
print("Oxygen1:", oxygen3,"umol/kg")
print("Oxygen2:", oxygen4,"umol/kg")

Oxygen1: [8.12350182] ml/l
Oxygen2: [8.12986258] ml/l
Oxygen1: [358.3635196] umol/kg
Oxygen2: [358.64431987] umol/kg


In [13]:
pressure_test = 1.197
deg = 54
minutes = 9.31
latitude = deg + minutes / 60
depth = depth_from_pressure(np.array(pressure_test),latitude)
depth

np.float64(1.1862956820496933)

In [14]:
time_elapsed = Time_elapsed(hexstrings)
print("Time_Elapsed:",time_elapsed,"s")

Time_Elapsed: [0.0, 0.041666666666666664, 0.08333333333333333, 0.125, 0.16666666666666666, 0.20833333333333331, 0.25, 0.29166666666666663, 0.3333333333333333, 0.375] s


In [15]:
print("Pressure:", pressure,"db")
print("Temp_1:",temperature1,"°C")
print("Temp_2:",temperature2,"°C")
print("Conductivity_1:", conductifity1,"mS/cm")
print("Conductivity_2:", conductifity2,"mS/cm")
print("Oxygen1:", oxygen1,"ml/l")
print("Oxygen2:", oxygen2,"ml/l")
print("Oxygen1:", oxygen3,"umol/kg")
print("Oxygen2:", oxygen4,"umol/kg")
print("Time_Elapsed:", ', '.join(f"{time_elapsed:.3f}" for time_elapsed in time_elapsed), "s")
print("Salinity1:", salinity1,"PSU")
print("Salinity2:", salinity2,"PSU")

Pressure: -42139.332086015915 db
Temp_1: 3.561563969710619 °C
Temp_2: 3.561044266345732 °C
Conductivity_1: 15.330789300015969 mS/cm
Conductivity_2: 15.32989468343113 mS/cm
Oxygen1: [8.12350182] ml/l
Oxygen2: [8.12986258] ml/l
Oxygen1: [358.3635196] umol/kg
Oxygen2: [358.64431987] umol/kg
Time_Elapsed: 0.000, 0.042, 0.083, 0.125, 0.167, 0.208, 0.250, 0.292, 0.333, 0.375 s
Salinity1: 15.5521222127295 PSU
Salinity2: 15.551386401977785 PSU


In [16]:
name 0 = Pressure: -42139.332086015915 db                        --> Falsch
name 1 = Temp_1: 3.561563969710619 °C                     --> Richtig
name 2 = Temp_2: 3.561044266345732 °C                     --> Richtig
name 3 = Conductivity_1: 15.330789300015969 mS/cm         --> Richtig
name 4 = Conductivity_2: 15.32989468343113 mS/cm          --> Richtig
name 5 = Oxygen1: [8.12350182] ml/l                       --> Richtig
name 6 = Oxygen2: [8.12986258] ml/l                       --> Richtig
name 7 = Oxygen1: [358.3635196] umol/kg                   --> Richtig
name 8 = Oxygen2: [358.64431987] umol/kg                  --> Richtig

name 9 = flECO-AFL: Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]   --> Noch nicht
name 10 = turbWETntu0: Turbidity, WET Labs ECO [NTU]             --> Noch nicht
name 11 = par: PAR/Irradiance, Biospherical/Licor                --> Noch nicht
name 12 = spar: SPAR, Biospherical/Licor                         --> Noch nicht
name 13 = timeS: Time, Elapsed [seconds]                  --> Richtig
name 14 = dz/dtM: Descent Rate [m/s]                             --> Later

name 15 = Salinity1: 15.5521222127295 PSU                 --> Richtig        
name 16 = Salinity2: 15.551386401977785 PSU               --> Richtig
name 17 = flag:  0.000e+00                                --> ???






SyntaxError: invalid character '°' (U+00B0) (803144344.py, line 2)